In [1]:
from google.colab import drive, output
drive.mount('/content/drive')
import sys
%cd '/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project'
sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/VQVAE_Working/data')
sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/VQVAE_Working/models')
sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Dataloader')
sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Models')

%load_ext autoreload
%autoreload 1
!pip install torchaudio
!pip install wandb --upgrade
!wandb login
output.clear()

In [2]:
from __future__ import print_function
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
#from scipy.signal import savgol_filter

from six.moves import xrange

#import umap
import wandb
import torch
#import pandas as pd

from scipy import signal
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim

from torch.utils.data import random_split
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.utils import make_grid
import torch.nn.functional as F

#from ResidualStack import ResidualStack

from Models import Model
from Dataloader import EcoData


from Models import Encoder

from Models import Decoder
from Residual import Residual
from Models import VectorQuantizer
from Models import VectorQuantizerEMA

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = xm.xla_device()
print(device)


cuda


In [3]:
wandb.login()

wandb: Currently logged in as: danielnieto (use `wandb login --relogin` to force relogin)


True

## Train

We use the hyperparameters from the author's code:

In [ ]:
root_path = '/content/drive/Shareddrives/ConservacionBiologicaIA/Datos'
labels_path = '/content/drive/Shareddrives/ConservacionBiologicaIA/Datos/Acceso_Datos_Humboldt/ensayo.xlsx'
names_path = '/content/drive/Shareddrives/ConservacionBiologicaIA/Datos/Acceso_Datos_Humboldt/RecordingsGuajiraTesisWeights.xlsx'

dataset = EcoData(root_path, labels_path, names_path, 60, "wav")
dataset_train, dataset_test = random_split(dataset,
                                           [round(len(dataset)*0.70), len(dataset) - round(len(dataset)*0.70)], 
                                           generator=torch.Generator().manual_seed(1024))

config = {
    "batch_size" : 2,
    "num_epochs": 20,
    "num_training_updates" : len(dataset_train),
    "num_hiddens" : 64,
    "embedding_dim" : 128,
    "num_embeddings" : 64,
    "commitment_cost" : 0.25,
    "decay" : 0.99,
    "learning_rate" : 1e-4,
    "dataset": "Audios Humboldt",
    "architecture": "VQ-VAE",
}

training_loader = DataLoader(dataset_train, batch_size=config["batch_size"])
test_loader = DataLoader(dataset_test, batch_size=config["batch_size"])


model = Model(config["num_hiddens"],
              config["num_embeddings"], config["embedding_dim"], 
              config["commitment_cost"], config["decay"]).to(device)

optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"], amsgrad=False)


wandb.finish()
wandb.init(project="VQ-VAE", config=config)


/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [ ]:
def testModel(model, iterator):
    model.eval()
    (valid_originals, _,_,_) = next( iterator)
    valid_originals = torch.reshape(valid_originals, (valid_originals.shape[0] * valid_originals.shape[1], 
                                                  valid_originals.shape[2], valid_originals.shape[3]))
    valid_originals = torch.unsqueeze(valid_originals,1)

    valid_originals = valid_originals.to(device)

    vq_output_eval = model._pre_vq_conv(model._encoder(valid_originals))
    _, valid_quantize, _, _ = model._vq_vae(vq_output_eval)
    valid_reconstructions = model._decoder(valid_quantize)
    output =  torch.cat((valid_originals, valid_reconstructions), 0)
    img_grid = make_grid(output,nrow= 5,pad_value=20)

    recon_error = F.mse_loss(valid_originals, valid_reconstructions)

    fig, ax = plt.subplots(figsize=(20,5))
    ax.imshow(img_grid[1,:,:].cpu(), vmin=0,vmax=1)
    ax.axis("off")
    return fig, recon_error


In [ ]:
model.train()
train_res_recon_error = []
train_res_perplexity = []
iterator = iter(test_loader)
wandb.watch(model, F.mse_loss, log="all", log_freq=1)

for epoch in range(config["num_epochs"]):
  for i in xrange(config["num_training_updates"]):
      (data, _,_,_) = next(iter(training_loader))
      #print(i)
      data = torch.reshape(data, (data.shape[0] * data.shape[1], data.shape[2], data.shape[3]))
      data = torch.unsqueeze(data,1)
      data = data.to(device)
      
      
      optimizer.zero_grad()
      vq_loss, data_recon, perplexity = model(data)
      
      recon_error = F.mse_loss(data_recon, data) #/ data_variance
      loss = recon_error + vq_loss
      loss.backward()

      optimizer.step()

      wandb.log({"loss":loss.item(),
                 "perplexity":perplexity.item(),
                 "recon_error": recon_error,
                 "vq_loss": vq_loss})
      print(f'epoch: {epoch} of {config["num_epochs"]} \t iteration: {(i+1)} of {config["num_training_updates"]} \t loss: {np.round(loss.item(),4)} \t recon_error: {np.round(recon_error.item(),4)} \t vq_loss: {np.round(vq_loss.item(),4)}')
      torch.cuda.empty_cache() 
      
      if (i+1) % 500 == 0:
        torch.save(model.state_dict(),f'model_{epoch}_{i}.pkl')
        fig, test_error = testModel(model, iterator)
        images = wandb.Image(fig, caption= f"recon_error: {np.round(test_error.item(),4)}")
        wandb.log({"examples": images})
                  
wandb.finish()


In [ ]:
model.load_state_dict(torch.load('model_1_12999_5s.pkl', map_location=torch.device('cpu')))

## Plot Loss

In [ ]:
train_res_recon_error_smooth = savgol_filter(train_res_recon_error, 201, 7)
train_res_perplexity_smooth = savgol_filter(train_res_perplexity, 201, 7)

In [ ]:
f = plt.figure(figsize=(16,8))
ax = f.add_subplot(1,2,1)
ax.plot(train_res_recon_error_smooth)
ax.set_yscale('log')
ax.set_title('Smoothed NMSE.')
ax.set_xlabel('iteration')

ax = f.add_subplot(1,2,2)
ax.plot(train_res_perplexity_smooth)
ax.set_title('Smoothed Average codebook usage (perplexity).')
ax.set_xlabel('iteration')

In [ ]:
model.eval()
validation_loader = training_loader
(valid_originals, _,_,_) = next(iter(validation_loader),30000)

## View Reconstructions

In [ ]:
model.eval()
validation_loader = training_loader
(valid_originals, _,_,_) = next(iter(validation_loader),3000)
valid_originals = torch.reshape(valid_originals, (valid_originals.shape[0] * valid_originals.shape[1], valid_originals.shape[2], valid_originals.shape[3]))
valid_originals = torch.unsqueeze(valid_originals,1)

print(valid_originals.shape)



valid_originals = valid_originals.to(device)

vq_output_eval = model._pre_vq_conv(model._encoder(valid_originals))
_, valid_quantize, _, _ = model._vq_vae(vq_output_eval)
valid_reconstructions = model._decoder(valid_quantize)

In [ ]:
def showTensor(tensor, dim):
    return tensor[dim,:,:,:].cpu().detach().numpy()[0,:,:]

In [ ]:
plt.figure(figsize=(40,40))

plt.subplot(5,2,1)
plt.imshow(showTensor(valid_originals,0),vmin=0,vmax=0.7)
plt.subplot(5,2,2)
plt.imshow(showTensor(valid_reconstructions,0),vmin=0,vmax=0.7)
plt.subplot(5,2,3)
plt.imshow(showTensor(valid_originals,1),vmin=0,vmax=0.7)
plt.subplot(5,2,4)
plt.imshow(showTensor(valid_reconstructions,1),vmin=0,vmax=0.7)
plt.subplot(5,2,5)
plt.imshow(showTensor(valid_originals,2),vmin=0,vmax=0.7)
plt.subplot(5,2,6)
plt.imshow(showTensor(valid_reconstructions,2),vmin=0,vmax=0.7)
plt.subplot(5,2,7)
plt.imshow(showTensor(valid_originals,3),vmin=0,vmax=0.7)
plt.subplot(5,2,8)
plt.imshow(showTensor(valid_reconstructions,3),vmin=0,vmax=0.7)
plt.subplot(5,2,9)
plt.imshow(showTensor(valid_originals,4),vmin=0,vmax=0.7)
plt.subplot(5,2,10)
plt.imshow(showTensor(valid_reconstructions,4),vmin=0,vmax=0.7)
#plt.savefig("VQVAE_Sin_Residual.pdf")

In [ ]:
(train_originals, _,_,_) = next(iter(training_loader))
train_originals = train_originals.to(device)
_, train_reconstructions, _, _ = model._vq_vae(train_originals)

In [ ]:
def show(img):
    npimg = img.numpy()
    fig = plt.imshow(np.transpose(npimg, (1,2,0)), interpolation='nearest')
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)

In [ ]:
show(make_grid(valid_reconstructions.cpu().data)+0.5, )

In [ ]:
show(make_grid(valid_originals.cpu()+0.5))

## View Embedding

In [ ]:
proj = umap.UMAP(n_neighbors=3,
                 min_dist=0.1,
                 metric='cosine').fit_transform(model._vq_vae._embedding.weight.data.cpu())

In [ ]:
plt.scatter(proj[:,0], proj[:,1], alpha=0.3)